# Optimizing a New Food Truck Business in San Diego, CA

In [2]:
#Please Note: The content of this notebook is drawn from the IBM Data Science Certification Course through Coursera

#Begin by importing all necessary python libraries

import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

print('Libraries imported!')

     |████████████████████████████████| 98 kB 9.9 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 6.4 MB/s  eta 0:00:01
Libraries imported!


In [3]:
#Auto-generated code to import csv file showing zip codes of San Diego county with corresponding cities and regions

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


body = client_898a3789eb53423e8b3dd654559a9bb5.get_object(Bucket='courseracapstoneproject-donotdelete-pr-q47qdqpf5g97mw',Key='SDZipCodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_zipcodes = pd.read_csv(body)
df_zipcodes.head()
body = client_898a3789eb53423e8b3dd654559a9bb5.get_object(Bucket='courseracapstoneproject-donotdelete-pr-q47qdqpf5g97mw',Key='SDZipCodes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_zipcodes = pd.read_csv(body)

#Show first five rows of imported csv file in newly created dataframe
df_zipcodes.head()


,Zip Code,City,Region
0,91901,Alpine,EAST
1,91902,Bonita,SOUTH
2,91903,Alpine,EAST
3,91905,Boulevard,EAST
4,91906,Campo,EAST


In [4]:
#Pull out zip code column and convert to list
zipCodes = df_zipcodes['Zip Code'].tolist()
zipCodes

[91901,
 91902,
 91903,
 91905,
 91906,
 91908,
 91909,
 91910,
 91911,
 91912,
 91913,
 91914,
 91915,
 91916,
 91917,
 91921,
 91931,
 91932,
 91933,
 91934,
 91935,
 91941,
 91942,
 91943,
 91944,
 91945,
 91948,
 91950,
 91951,
 91962,
 91963,
 91977,
 91978,
 91979,
 91980,
 91987,
 92003,
 92004,
 92007,
 92008,
 92009,
 92010,
 92011,
 92013,
 92014,
 92018,
 92019,
 92020,
 92021,
 92022,
 92023,
 92024,
 92025,
 92026,
 92027,
 92028,
 92029,
 92030,
 92033,
 92036,
 92037,
 92039,
 92040,
 92046,
 92049,
 92051,
 92052,
 92054,
 92055,
 92056,
 92057,
 92059,
 92060,
 92061,
 92064,
 92065,
 92066,
 92067,
 92069,
 92070,
 92071,
 92072,
 92075,
 92078,
 92079,
 92081,
 92082,
 92083,
 92084,
 92085,
 92086,
 92088,
 92091,
 92092,
 92093,
 92096,
 92101,
 92102,
 92103,
 92104,
 92105,
 92106,
 92107,
 92108,
 92109,
 92110,
 92111,
 92112,
 92113,
 92114,
 92115,
 92116,
 92117,
 92118,
 92119,
 92120,
 92121,
 92122,
 92123,
 92124,
 92126,
 92127,
 92128,
 92129,
 92130,


In [5]:
#create empty lists for latitudes and longitudes of corresponding zip codes
latitudes = []
longitudes = []

#use Nominatim to retrieve latitudes and longitudes
#for loop to populate latitudes and longitudes lists as each zip code is queried
for zcode in zipCodes:
    geolocator = Nominatim(user_agent="my_request")
    location = geolocator.geocode(zcode)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)

In [6]:
#latitude and longitudes lists added to dataframe
df_zipcodes['Latitude'] = latitudes
df_zipcodes['Longitude'] = longitudes
#show first 5 rows of new dataframe
df_zipcodes.head()

,Zip Code,City,Region,Latitude,Longitude
0,91901,Alpine,EAST,48.409197,17.493948
1,91902,Bonita,SOUTH,32.666379,-117.022725
2,91903,Alpine,EAST,48.905857,24.713281
3,91905,Boulevard,EAST,32.662786,-116.312998
4,91906,Campo,EAST,32.662691,-116.481502


In [7]:
#drop zip code column since it is no longer needed
df_sandiego = df_zipcodes.drop(columns='Zip Code')
df_sandiego.head()

,City,Region,Latitude,Longitude
0,Alpine,EAST,48.409197,17.493948
1,Bonita,SOUTH,32.666379,-117.022725
2,Alpine,EAST,48.905857,24.713281
3,Boulevard,EAST,32.662786,-116.312998
4,Campo,EAST,32.662691,-116.481502


In [8]:
#show latitude and longitude of San Diego
address = 'San Diego, CA'

geolocator = Nominatim(user_agent = "sandiego_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinates of San Diego are {}, {}.'.format(latitude, longitude))

The geographical coordinates of San Diego are 32.7174202, -117.1627728.


In [9]:
#show map of San Diego with city, region labels
map_SanDiego = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, region, city in zip(df_sandiego['Latitude'], df_sandiego['Longitude'], df_sandiego['Region'], df_sandiego['City']):
    label = '{}, {}'.format(city, region)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SanDiego)

map_SanDiego

In [11]:
#function to query Foursquare and retrieve venues based on latitude and longitude
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
#executing the function for San Diego
sandiego_venues = getNearbyVenues(df_sandiego['City'], latitudes=df_sandiego['Latitude'], longitudes=df_sandiego['Longitude'])

Alpine
Bonita
Alpine
Boulevard
Campo
Bonita
Chula Vista
Chula Vista
Chula Vista
Chula Vista
Chula Vista
Chula Vista
Chula Vista
Descanso
Dulzura
Chula Vista
Guatay
Imperial Beach
Imperial Beach
Jacumba
Jamul
La Mesa
La Mesa
La Mesa
La Mesa
Lemon Grove
Mount Laguna
National City
National City
Pine Valley
Potrero
Spring Valley
Spring Valley
Spring Valley
Tecate
Tecate
Bonsall
Borrego Springs/Ocotillo Wells
Cardiff By The Sea
Carlsbad
Carlsbad
Carlsbad
Carlsbad
Carlsbad
Del Mar
Carlsbad
El Cajon
El Cajon
El Cajon
El Cajon
Encinitas
Encinitas
Escondido
Escondido
Escondido
Fallbrook
Escondido
Escondido
Escondido
Julian
La Jolla
La Jolla
Lakeside
Escondido
Oceanside
Oceanside
Oceanside
Oceanside
Camp Pendleton
Oceanside
Oceanside
Pala
Palomar Mountain
Pauma Valley
Poway
Ramona
Ranchita
Rancho Santa Fe
San Marcos
Santa Ysabel
Santee
Santee
Solana Beach
San Marcos
San Marcos
Vista
Valley Center
Vista
Vista
Vista
Warner Springs
Fallbrook
Rancho Santa Fe
La Jolla
La Jolla
San Marcos
San Diego, D

In [13]:
#showing the number of columns and rows of the new dataframe
print(sandiego_venues.shape)
#showing the first five rows of the new dataframe
sandiego_venues.head()

(1376, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alpine,48.409197,17.493948,Restaurant & Pizza ŠRÁMEK,48.409530,17.495751,Gastropub
1,Alpine,48.409197,17.493948,Piváreň Zákruta,48.408316,17.492037,Bar
2,Bonita,32.666379,-117.022725,Starbucks,32.669177,-117.022423,Coffee Shop
3,Bonita,32.666379,-117.022725,Hans & Harry Bakery,32.670406,-117.021200,Bakery
4,Bonita,32.666379,-117.022725,Club Caribe,32.670370,-117.021281,Nightclub


In [14]:
#showing a count of venues by neighborhood
sandiego_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alpine,18,18,18,18,18,18
Bonita,12,12,12,12,12,12
Borrego Springs/Ocotillo Wells,1,1,1,1,1,1
Camp Pendleton,46,46,46,46,46,46
Cardiff By The Sea,4,4,4,4,4,4
...,...,...,...,...,...,...
Santee,49,49,49,49,49,49
Solana Beach,32,32,32,32,32,32
Spring Valley,10,10,10,10,10,10


In [15]:
#determing the number of unique categories of venues
print('There are {} uniques categories.'.format(len(sandiego_venues['Venue Category'].unique())))

There are 258 uniques categories.


In [16]:
# one hot encoding to assign frequencies to each type of venue
sandiego_onehot = pd.get_dummies(sandiego_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sandiego_onehot['Neighborhood'] = sandiego_venues['Neighborhood'] 

#show first 5 rows of new dataframe
sandiego_onehot.head()

,ATM,Accessories Store,American Restaurant,Antique Shop,Apres Ski Bar,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#show the number of rows and columns of the onehot dataframe
sandiego_onehot.shape

(1376, 258)

In [18]:
#group by neighborhood to get average frequency
sandiego_grouped = sandiego_onehot.groupby('Neighborhood').mean().reset_index()
sandiego_grouped

,Neighborhood,ATM,Accessories Store,American Restaurant,Antique Shop,Apres Ski Bar,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,...,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Alpine,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
1,Bonita,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.083333,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.083333
2,Borrego Springs/Ocotillo Wells,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,Camp Pendleton,0.000000,0.021739,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,Cardiff By The Sea,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,Santee,0.000000,0.000000,0.0,0.0,0.0,0.0,0.020408,0.020408,0.000000,...,0.020408,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.020408
63,Solana Beach,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.031250,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
64,Spring Valley,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000
65,Tecate,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000


In [19]:
#show number of rows and columns of grouped dataframe
sandiego_grouped.shape

(67, 258)

In [20]:
#funtion to return the most common venues from Foursquare data
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
#number of top venues set to 20
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sandiego_grouped['Neighborhood']

for ind in np.arange(sandiego_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sandiego_grouped.iloc[ind, :], num_top_venues)

#show the top 5 rows of newly created dataframe
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Alpine,Gastropub,Bar,Gym,Bus Stop,Shopping Mall,Fried Chicken Joint,Farmers Market,Supermarket,Furniture / Home Store,...,Bus Station,Restaurant,Grocery Store,Food & Drink Shop,Discount Store,Athletics & Sports,Pharmacy,Dessert Shop,Dance Studio,Flower Shop
1,Bonita,Yoga Studio,Discount Store,Fried Chicken Joint,Shopping Plaza,Taco Place,Nightclub,Gastropub,Bakery,Coffee Shop,...,Arts & Entertainment,Pizza Place,Diner,Farm,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Dance Studio,Fishing Store
2,Borrego Springs/Ocotillo Wells,Furniture / Home Store,Yoga Studio,Football Stadium,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fishing Store,...,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Electronics Store,Donut Shop,Doner Restaurant,Dog Run,Dive Bar
3,Camp Pendleton,Supermarket,Coffee Shop,Sandwich Place,Clothing Store,Furniture / Home Store,Cosmetics Shop,Bakery,Plaza,Sporting Goods Shop,...,Frozen Yogurt Shop,Bookstore,Electronics Store,Fountain,Roof Deck,Food Court,Salad Place,Salon / Barbershop,Mexican Restaurant,Shopping Plaza
4,Cardiff By The Sea,Hotel,Peruvian Restaurant,Mexican Restaurant,Intersection,Financial or Legal Service,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Fishing Store,Electronics Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Doner Restaurant,Donut Shop,Football Stadium


In [22]:
# set number of clusters to 20
kclusters = 20

#drop the neighborhood column
sandiego_grouped_clustering = sandiego_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sandiego_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([17, 17,  4, 17, 10, 17, 17, 10,  2, 17], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sandiego_merged = df_sandiego

# merge sandiego_grouped with sandiego_data to add latitude/longitude for each neighborhood
sandiego_merged = sandiego_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City')

#drop any NaN values
sandiego_merged.dropna(axis=0, inplace=True)
#cast Cluster Labels column to int
sandiego_merged = sandiego_merged.astype({"Cluster Labels": 'int'})
#show first 5 rows of data in new dataframe
sandiego_merged.head()

,City,Region,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Alpine,EAST,48.409197,17.493948,17,Gastropub,Bar,Gym,Bus Stop,Shopping Mall,...,Bus Station,Restaurant,Grocery Store,Food & Drink Shop,Discount Store,Athletics & Sports,Pharmacy,Dessert Shop,Dance Studio,Flower Shop
1,Bonita,SOUTH,32.666379,-117.022725,17,Yoga Studio,Discount Store,Fried Chicken Joint,Shopping Plaza,Taco Place,...,Arts & Entertainment,Pizza Place,Diner,Farm,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Dance Studio,Fishing Store
2,Alpine,EAST,48.905857,24.713281,17,Gastropub,Bar,Gym,Bus Stop,Shopping Mall,...,Bus Station,Restaurant,Grocery Store,Food & Drink Shop,Discount Store,Athletics & Sports,Pharmacy,Dessert Shop,Dance Studio,Flower Shop
5,Bonita,SOUTH,48.461970,17.495524,17,Yoga Studio,Discount Store,Fried Chicken Joint,Shopping Plaza,Taco Place,...,Arts & Entertainment,Pizza Place,Diner,Farm,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Dance Studio,Fishing Store
6,Chula Vista,SOUTH,48.929178,24.709396,17,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sandiego_merged['Latitude'], sandiego_merged['Longitude'], sandiego_merged['City'], sandiego_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
#cluster 1
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 0, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
31,Spring Valley,Pool,Home Service,Seafood Restaurant,Massage Studio,Park,Discount Store,Electronics Store,Fabric Shop,Factory,...,Hobby Shop,Donut Shop,Fast Food Restaurant,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Farmers Market,Doner Restaurant
32,Spring Valley,Pool,Home Service,Seafood Restaurant,Massage Studio,Park,Discount Store,Electronics Store,Fabric Shop,Factory,...,Hobby Shop,Donut Shop,Fast Food Restaurant,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Farmers Market,Doner Restaurant
33,Spring Valley,Pool,Home Service,Seafood Restaurant,Massage Studio,Park,Discount Store,Electronics Store,Fabric Shop,Factory,...,Hobby Shop,Donut Shop,Fast Food Restaurant,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Farmers Market,Doner Restaurant
124,"San Diego, Carmel Valley",Park,Pool,American Restaurant,Dog Run,Financial or Legal Service,Fabric Shop,Factory,Farm,Farmers Market,...,Yoga Studio,Electronics Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fishing Store,Doner Restaurant,Donut Shop


In [26]:
#cluster 2
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 1, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
74,Poway,Construction & Landscaping,Yoga Studio,Electronics Store,Food Truck,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Donut Shop,Cultural Center,Doner Restaurant,Dog Run


In [27]:
#cluster 3
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 2, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
44,Del Mar,Bar,Beach,Business Service,Italian Restaurant,Shopping Mall,Fishing Store,Farm,Farmers Market,Fast Food Restaurant,...,Yoga Studio,Factory,Flower Shop,Food & Drink Shop,Food Court,Food Service,Flea Market,Fabric Shop,Football Stadium,Electronics Store
119,"San Diego, Tierrasanta",Park,Baseball Field,Trail,Basketball Court,Yoga Studio,Electronics Store,Factory,Farm,Farmers Market,...,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Donut Shop,Football Stadium
125,"San Diego, Scripps Ranch",Park,Home Service,Financial or Legal Service,Nightlife Spot,Yoga Studio,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Electronics Store,Donut Shop,Food Truck,Doner Restaurant,Dog Run
135,"San Diego, Tierrasanta",Park,Baseball Field,Trail,Basketball Court,Yoga Studio,Electronics Store,Factory,Farm,Farmers Market,...,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Donut Shop,Football Stadium


In [28]:
#cluster 4
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 3, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
34,Tecate,Café,Beach,Resort,Financial or Legal Service,Factory,Farm,Farmers Market,Fast Food Restaurant,Yoga Studio,...,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Donut Shop,Football Stadium,Doner Restaurant
35,Tecate,Café,Beach,Resort,Financial or Legal Service,Factory,Farm,Farmers Market,Fast Food Restaurant,Yoga Studio,...,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Donut Shop,Football Stadium,Doner Restaurant


In [29]:
#cluster 5
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 4, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
37,Borrego Springs/Ocotillo Wells,Furniture / Home Store,Yoga Studio,Football Stadium,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fishing Store,...,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Electronics Store,Donut Shop,Doner Restaurant,Dog Run,Dive Bar


In [30]:
#cluster 6
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 5, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
168,"San Diego, Rolando",Grocery Store,Department Store,Yoga Studio,Financial or Legal Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Donut Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Electronics Store,Doner Restaurant,Football Stadium,Dog Run


In [31]:
#cluster 7
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 6, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
30,Potrero,RV Park,Café,Yoga Studio,Electronics Store,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Donut Shop,Football Stadium,Doner Restaurant,Dog Run


In [32]:
#cluster 8
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 7, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
114,"San Diego, Navajo",Smoke Shop,Liquor Store,Yoga Studio,Electronics Store,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Donut Shop,Football Stadium,Doner Restaurant,Dog Run


In [33]:
#cluster 9
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 8, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
133,"San Diego, Paradise Hills",Baby Store,High School,Business Service,Night Market,Yoga Studio,Fishing Store,Farm,Farmers Market,Fast Food Restaurant,...,Flea Market,Fabric Shop,Flower Shop,Food & Drink Shop,Food Court,Food Service,Factory,Donut Shop,Electronics Store,Football Stadium


In [34]:
#cluster 10
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 9, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
20,Jamul,Business Service,Yoga Studio,Cultural Center,Food Truck,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Electronics Store,Donut Shop,Doner Restaurant,Dog Run


In [35]:
#cluster 11
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 10, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
38,Cardiff By The Sea,Hotel,Peruvian Restaurant,Mexican Restaurant,Intersection,Financial or Legal Service,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Fishing Store,Electronics Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Doner Restaurant,Donut Shop,Football Stadium
79,Santa Ysabel,Hotel,Bakery,Food Truck,Bus Station,Fast Food Restaurant,Food Service,Food Court,Food & Drink Shop,Flower Shop,...,Fishing Store,Financial or Legal Service,Farmers Market,Farm,Factory,Fabric Shop,Electronics Store,Fountain,Donut Shop,Doner Restaurant
113,Coronado,Hotel,Harbor / Marina,Mexican Restaurant,Seafood Restaurant,Beach,Gift Shop,Factory,Farm,Farmers Market,...,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Electronics Store,Football Stadium
116,"San Diego, Sorrento Valley",Hotel,Tex-Mex Restaurant,Sushi Restaurant,Playground,Deli / Bodega,Mexican Restaurant,Restaurant,Café,Factory,...,Farmers Market,Yoga Studio,Fast Food Restaurant,Financial or Legal Service,Fabric Shop,Flea Market,Flower Shop,Food & Drink Shop,Fishing Store,Doner Restaurant
159,Coronado,Hotel,Harbor / Marina,Mexican Restaurant,Seafood Restaurant,Beach,Gift Shop,Factory,Farm,Farmers Market,...,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Electronics Store,Football Stadium
164,"San Diego, Sorrento Valley",Hotel,Tex-Mex Restaurant,Sushi Restaurant,Playground,Deli / Bodega,Mexican Restaurant,Restaurant,Café,Factory,...,Farmers Market,Yoga Studio,Fast Food Restaurant,Financial or Legal Service,Fabric Shop,Flea Market,Flower Shop,Food & Drink Shop,Fishing Store,Doner Restaurant


In [36]:
#cluster 12
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 11, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
106,"San Diego, Linda Vista",Japanese Restaurant,Dog Run,Fishing Store,Factory,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Yoga Studio,...,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fabric Shop,Donut Shop,Fountain,Doner Restaurant
152,"San Diego, Linda Vista",Japanese Restaurant,Dog Run,Fishing Store,Factory,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Yoga Studio,...,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fabric Shop,Donut Shop,Fountain,Doner Restaurant


In [37]:
#cluster 13
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 12, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
101,"San Diego, Point Loma",Coffee Shop,Supermarket,Mexican Restaurant,Yoga Studio,Financial or Legal Service,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Fishing Store,Electronics Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Donut Shop,Football Stadium,Doner Restaurant
149,"San Diego, Point Loma",Coffee Shop,Supermarket,Mexican Restaurant,Yoga Studio,Financial or Legal Service,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Fishing Store,Electronics Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,Fabric Shop,Donut Shop,Football Stadium,Doner Restaurant


In [38]:
#cluster 14
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 13, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
102,"San Diego, Ocean Beach",Park,Botanical Garden,Construction & Landscaping,Fishing Store,Factory,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,...,Flea Market,Electronics Store,Food & Drink Shop,Food Court,Food Service,Food Truck,Fabric Shop,Yoga Studio,Fountain,Donut Shop
109,"San Diego, Encanto",Park,Construction & Landscaping,Bakery,Financial or Legal Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Fishing Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Electronics Store,Yoga Studio,Fountain,Donut Shop
155,"San Diego, Encanto",Park,Construction & Landscaping,Bakery,Financial or Legal Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Fishing Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Electronics Store,Yoga Studio,Fountain,Donut Shop


In [39]:
#cluster 15
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 14, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
25,Lemon Grove,Convenience Store,Home Service,Liquor Store,Fried Chicken Joint,Pizza Place,Mexican Restaurant,Light Rail Station,BBQ Joint,Fast Food Restaurant,...,Farm,Farmers Market,Flea Market,Financial or Legal Service,Fishing Store,Flower Shop,Food & Drink Shop,Food Court,Fabric Shop,Donut Shop
62,Lakeside,Mexican Restaurant,Pizza Place,Italian Restaurant,Bar,Sandwich Place,Park,German Restaurant,Discount Store,Grocery Store,...,Donut Shop,Gym / Fitness Center,Hardware Store,Ice Cream Shop,Fishing Store,Liquor Store,Farmers Market,Fast Food Restaurant,Farm,Factory
77,Rancho Santa Fe,Golf Course,American Restaurant,BBQ Joint,Café,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Fishing Store,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Food Truck,Farm,Factory,Yoga Studio,Electronics Store,Fountain
82,Solana Beach,Pizza Place,Ice Cream Shop,Sandwich Place,Grocery Store,Bakery,Coffee Shop,Liquor Store,Supplement Shop,Spa,...,Gas Station,Mexican Restaurant,Burger Joint,Paper / Office Supplies Store,Juice Bar,Bank,Bagel Shop,Fast Food Restaurant,Food Service,Cosmetics Shop
92,Rancho Santa Fe,Golf Course,American Restaurant,BBQ Joint,Café,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Fishing Store,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Food Truck,Farm,Factory,Yoga Studio,Electronics Store,Fountain
108,"San Diego, Logan Heights",Sandwich Place,Grocery Store,Intersection,Pizza Place,Harbor / Marina,Fast Food Restaurant,Electronics Store,Fabric Shop,Factory,...,Farmers Market,Yoga Studio,Donut Shop,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Financial or Legal Service,Dog Run
110,"San Diego, College Grove",Sandwich Place,Pizza Place,BBQ Joint,Bubble Tea Shop,Juice Bar,Salon / Barbershop,Coffee Shop,Frozen Yogurt Shop,Sporting Goods Shop,...,Fast Food Restaurant,Bank,Restaurant,Burger Joint,Mexican Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Café,Mobile Phone Shop,Japanese Restaurant
117,"San Diego, University",Sandwich Place,Bus Stop,Park,Gas Station,Recreation Center,Flea Market,Fishing Store,Financial or Legal Service,Donut Shop,...,Farmers Market,Farm,Flower Shop,Factory,Fabric Shop,Food & Drink Shop,Electronics Store,Yoga Studio,Doner Restaurant,Food Service
136,San Ysidro,Shipping Store,Donut Shop,Chinese Restaurant,Grocery Store,Fast Food Restaurant,Sandwich Place,Factory,Electronics Store,Fabric Shop,...,Farm,Financial or Legal Service,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Farmers Market,Doner Restaurant,Food Service,Dog Run
142,"San Diego, Otay Mesa",Mexican Restaurant,Park,Market,Gym,Fried Chicken Joint,Gas Station,Video Store,Recreation Center,Pizza Place,...,Fast Food Restaurant,Financial or Legal Service,Fishing Store,Farmers Market,Farm,Flea Market,Factory,Fabric Shop,Electronics Store,Donut Shop


In [40]:
#cluster 16
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 15, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
55,Fallbrook,Comfort Food Restaurant,Basketball Court,Construction & Landscaping,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fishing Store,...,Fabric Shop,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Factory,Donut Shop,Electronics Store,Fountain
91,Fallbrook,Comfort Food Restaurant,Basketball Court,Construction & Landscaping,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fishing Store,...,Fabric Shop,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Factory,Donut Shop,Electronics Store,Fountain


In [41]:
#cluster 17
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 16, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
141,"San Diego, Nestor",Fast Food Restaurant,Seafood Restaurant,Harbor / Marina,Yoga Studio,Fabric Shop,Factory,Farm,Farmers Market,Financial or Legal Service,...,Fishing Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Electronics Store,Doner Restaurant,Football Stadium,Dog Run


In [45]:
#cluster 18
#code to show all rows of this cluster
pd.set_option('display.max_rows', None)
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 17, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Alpine,Gastropub,Bar,Gym,Bus Stop,Shopping Mall,Fried Chicken Joint,Farmers Market,Supermarket,Furniture / Home Store,...,Bus Station,Restaurant,Grocery Store,Food & Drink Shop,Discount Store,Athletics & Sports,Pharmacy,Dessert Shop,Dance Studio,Flower Shop
1,Bonita,Yoga Studio,Discount Store,Fried Chicken Joint,Shopping Plaza,Taco Place,Nightclub,Gastropub,Bakery,Coffee Shop,...,Arts & Entertainment,Pizza Place,Diner,Farm,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Dance Studio,Fishing Store
2,Alpine,Gastropub,Bar,Gym,Bus Stop,Shopping Mall,Fried Chicken Joint,Farmers Market,Supermarket,Furniture / Home Store,...,Bus Station,Restaurant,Grocery Store,Food & Drink Shop,Discount Store,Athletics & Sports,Pharmacy,Dessert Shop,Dance Studio,Flower Shop
5,Bonita,Yoga Studio,Discount Store,Fried Chicken Joint,Shopping Plaza,Taco Place,Nightclub,Gastropub,Bakery,Coffee Shop,...,Arts & Entertainment,Pizza Place,Diner,Farm,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Dance Studio,Fishing Store
6,Chula Vista,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,Clothing Store,Furniture / Home Store,Electronics Store,Supermarket,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop
7,Chula Vista,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,Clothing Store,Furniture / Home Store,Electronics Store,Supermarket,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop
8,Chula Vista,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,Clothing Store,Furniture / Home Store,Electronics Store,Supermarket,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop
9,Chula Vista,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,Clothing Store,Furniture / Home Store,Electronics Store,Supermarket,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop
10,Chula Vista,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,Clothing Store,Furniture / Home Store,Electronics Store,Supermarket,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop
11,Chula Vista,Shopping Mall,Park,Cosmetics Shop,Pizza Place,Café,Clothing Store,Furniture / Home Store,Electronics Store,Supermarket,...,Pub,Cupcake Shop,Perfume Shop,Sporting Goods Shop,Bistro,Football Stadium,Soccer Field,Bowling Alley,Food & Drink Shop,Mobile Phone Shop


In [43]:
#cluster 19
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 18, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
26,Mount Laguna,Park,Breakfast Spot,Inn,Financial or Legal Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,...,Electronics Store,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fishing Store,Donut Shop,Doner Restaurant


In [44]:
#cluster 20
sandiego_merged.loc[sandiego_merged['Cluster Labels'] == 19, sandiego_merged.columns[[0] + list(range(5, sandiego_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
16,Guatay,Bus Station,Café,Yoga Studio,Football Stadium,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,...,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Electronics Store,Donut Shop,Doner Restaurant,Dog Run
